# **Problem 6: Solving a Jigsaw Puzzle Using Basic Image Operations**

In [1]:
import cv2
import numpy as np
import os

# A.

In [2]:
if not os.path.exists("pieces"):
    os.makedirs("pieces")

image = cv2.imread("Q6.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect pieces from the white background
_, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

pieces = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]

# Sort pieces from top to bottom and left to right
pieces.sort(key=lambda cnt: (cv2.boundingRect(cnt)[1], cv2.boundingRect(cnt)[0]))

for i, cnt in enumerate(pieces):
    x, y, w, h = cv2.boundingRect(cnt)
    # Crop the piece from the image
    piece = image[y:y+h, x:x+w]
    cv2.imwrite(f"pieces/piece_{i+1:02d}.jpg", piece)

print(f"{len(pieces)} pieces extracted and saved in the 'pieces' folder.")

15 pieces extracted and saved in the 'pieces' folder.


# B.

In [3]:
def rotate_piece(piece, angle, target_size=(250, 190)):
    if angle == 0:
        out = piece.copy()
    elif angle == 90:
        out = cv2.rotate(piece, cv2.ROTATE_90_CLOCKWISE)
    elif angle == 180:
        out = cv2.rotate(piece, cv2.ROTATE_180)
    elif angle == 270:
        out = cv2.rotate(piece, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return cv2.resize(out, target_size)

def calculate_edge_similarity(piece1, piece2, direction='right-left', edge_width=20):
    # Compare edges using pixel intensity differences
    if direction == 'right-left':
        edge1 = piece1[:, -edge_width:]
        edge2 = piece2[:, :edge_width]
    elif direction == 'left-right':
        edge1 = piece1[:, :edge_width]
        edge2 = piece2[:, -edge_width:]
    elif direction == 'bottom-top':
        edge1 = piece1[-edge_width:, :]
        edge2 = piece2[:edge_width, :]
    elif direction == 'top-bottom':
        edge1 = piece1[:edge_width, :]
        edge2 = piece2[-edge_width:, :]
    
    diff = np.abs(edge1.astype(np.float32) - edge2.astype(np.float32))
    return np.sum(diff)

def calculate_diagonal_similarity(piece1, piece2, edge_width=20):
    # Compare pixel differences at the corners of the pieces
    top_left_edge1 = piece1[:edge_width, :edge_width]
    bottom_right_edge2 = piece2[-edge_width:, -edge_width:]
    top_right_edge1 = piece1[:edge_width, -edge_width:]
    bottom_left_edge2 = piece2[-edge_width:, :edge_width]
    
    diff_top_left = np.abs(top_left_edge1.astype(np.float32) - bottom_right_edge2.astype(np.float32))
    diff_top_right = np.abs(top_right_edge1.astype(np.float32) - bottom_left_edge2.astype(np.float32))
    
    return np.sum(diff_top_left) + np.sum(diff_top_right)

# Load and resize puzzle pieces
target_size = (250, 190)
pieces = [cv2.resize(cv2.imread(f"pieces/piece_{i:02d}.jpg"), target_size)
          for i in range(1, 16)]

puzzle_matrix = np.zeros((5, 3), dtype=int)
puzzle_matrix[0, 0] = 1
remaining_pieces = set(range(2, 16))

# Placing puzzle pieces in the correct positions
for row in range(5):
    for col in range(3):
        if row == 0 and col == 0:
            continue 
        
        best = {'piece': None, 'similarity': float('inf'), 'angle': 0}
        neighbors = []

        if row > 0:
            neighbors.append(('top', puzzle_matrix[row-1, col]))
        if col > 0:
            neighbors.append(('left', puzzle_matrix[row, col-1]))
        
        # Check  all remaining pieces and their rotations
        for piece_num in remaining_pieces:
            for angle in [0, 90, 180, 270]:
                curr = rotate_piece(pieces[piece_num-1], angle)
                total_sim = 0
                
                # Calculate similarity with adjacent pieces
                for n_type, n_num in neighbors:
                    neighbor = pieces[n_num-1]
                    if n_type == 'top':
                        total_sim += calculate_edge_similarity(neighbor, curr, 'bottom-top')
                    elif n_type == 'left':
                        total_sim += calculate_edge_similarity(neighbor, curr, 'right-left')

                total_sim += calculate_diagonal_similarity(neighbor, curr)
                
                if total_sim < best['similarity']:
                    best = {'piece': piece_num, 'similarity': total_sim, 'angle': angle}
        
        puzzle_matrix[row, col] = best['piece']
        remaining_pieces.remove(best['piece'])
        # Update with the correct angle
        pieces[best['piece']-1] = rotate_piece(pieces[best['piece']-1], best['angle'])

print("Jigsaw Puzzle Matrix:")
print(puzzle_matrix)

Jigsaw Puzzle Matrix:
[[ 1  7  5]
 [14 12  4]
 [13  2 11]
 [ 8  3 15]
 [ 9 10  6]]


# C.

In [4]:
final_image = np.zeros((750, 954, 3), dtype=np.uint8)

piece_height = 750 // 5 
piece_width = 954 // 3   

# Placing pieces in the final image
for row in range(5):
    for col in range(3):
        y_start = row * piece_height
        y_end = y_start + piece_height
        x_start = col * piece_width
        x_end = x_start + piece_width
        
        piece_num = puzzle_matrix[row, col]
        piece_resized = cv2.resize(pieces[piece_num - 1], (piece_width, piece_height))
        # Placing piece in the final image
        final_image[y_start:y_end, x_start:x_end] = piece_resized

cv2.imshow("Completed Puzzle", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite("solved_puzzle.jpg", final_image)
print("The final image has been saved as 'solved_puzzle.jpg'.")

The final image has been saved as 'solved_puzzle.jpg'.
